# Mosaicing OPERA RTC Images

In this notebook, we will work with products from the Observational Products for End-Users from Remote Sensing Analysis (OPERA) project at the Jet Propolsion Labratory (JPL). OPERA data include Radiometric Terrain Correction (RTC) Synthetic Aperature Radar (SAR) images, which remove distortions by referencing the Copernicous global 30-m (GLO-30) Digital Elevation Model (DEM).

OPERA Products can be accessed using the Alaska Satellite Facility (ASF) SAR Data Search tools via the graphical interface [Vertex](https://search.asf.alaska.edu/#/), the [ASF API](https://docs.asf.alaska.edu/api/basics/) or using the  [`asf_search`](https://docs.asf.alaska.edu/asf_search/basics/) python module. If you do not already have one, please create a free [Earthdata Login Account](https://urs.earthdata.nasa.gov/users/new) to access products using ASF Search Tools.

In this module, we will access OPERA RTC images using the `asf_search` python module, download the selected bursts, and merge them to create a mosaic.

## 0. Check Conda environment to manage Python modules
If running in an OSL environment, run this cell to make sure you have the right conda environment set up! If running in Binder, please skip this cell.

In [ ]:
from IPython.display import Markdown
from IPython.display import display

env = !echo $CONDA_PREFIX
if env != '/home/joyvan/.local/envs/opera_rtc':
	display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "rtc_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select the "rtc_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "rtc_analysis" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))


## 1. Search for OPERA RTC products over San Francisco
After checking we have our environment correctly set up, we can perform the search for our desired products.The polygon in the search is centered over San Francisco.

In [ ]:
import asf_search

options = {
	'intersectsWith': 'POLYGON((-122.5306 37.6278,-122.1372 37.6207,-122.1372 37.9566,-122.5287 37.9554,-122.5306 37.6278))',
	'dataset': 'OPERA-S1',
	'start': '2023-11-14T14:00:00Z',
	'end': '2023-11-14T15:00:00Z',
	'flightDirection': 'DESCENDING',
	'processingLevel': [
		'RTC',
		'RTC-STATIC'
	],
	'maxResults': '1000'
}

results = asf_search.search(**options)
print(results)

## 2. Download selected bursts

In [ ]:
import os

data_directory = f'{os.getcwd()}/data/'
if not os.path.isdir(data_directory):
	os.mkdir(data_directory)

session = asf_search.ASFSession().auth_with_creds('user', 'pass')
results.download(path=data_directory, session=session)

## 3. Mosaic bursts and save

In [ ]:
import osgeo_utils.gdal_merge as gdal_merge

output_file_path = f'{os.getcwd()}/OPERA_RTC_San_Francisco.tif'
input_files_paths = [f'{data_directory}{path}' for path in os.listdir(data_directory)]
parameters = ['', '-o', output_file_path] + input_files_paths + ['-co', 'COMPRESS=LZW']
gdal_merge.main(parameters)